### Data preparation for supervised training

In [ ]:
import os
import zipfile
import shutil

from urllib.request import urlopen

In [ ]:
if os.name =='nt':
    # Windows
    os.environ['comspec'] = 'pwsh'            # use if cross platform PowerShell is installed
    # os.environ['comspec'] = 'powershell.exe'  # use default otherwise

In [ ]:
def unzip(path: str):
    with zipfile.ZipFile(path) as z:
        z.extractall()


def download(url: str, filepath: str):
    with urlopen(url) as page:
        with open(filepath, 'wb') as f:
            f.write(page.read())

In [ ]:
# place Adobe Matting Dataset zip here and run the cell

if os.path.exists('Adobe_Deep_Matting_Dataset.zip'):
    unzip('Adobe_Deep_Matting_Dataset.zip')

if os.path.exists('Adobe_Deep_Matting_Dataset.zip'):
    os.remove('Adobe_Deep_Matting_Dataset.zip')

In [ ]:
# create combined Dataset

def copy_adobe(source: str, target: str):
    os.mkdir(f'fg_{target}')
    os.mkdir(f'mask_{target}')
    os.mkdir(f'merged_{target}')
    with open(f'{target}_data_list.txt') as f:
        for line in f:
            line: str = line.strip()
            if os.path.isfile(f'{source}/Other/fg/{line}'):
                shutil.copyfile(f'{source}/Other/fg/{line}', f'fg_{target}/{line}')
                shutil.copyfile(f'{source}/Other/alpha/{line}', f'mask_{target}/{line}')
            else:
                shutil.copyfile(f'{source}/Adobe-licensed images/fg/{line}', f'fg_{target}/{line}')
                shutil.copyfile(f'{source}/Adobe-licensed images/alpha/{line}', f'mask_{target}/{line}')

In [ ]:
os.chdir('Data_Adobe')

In [ ]:
copy_adobe('../Combined_Dataset/Test_set', 'test')
copy_adobe('../Combined_Dataset/Training_set', 'train')

In [ ]:
# Getting MS-COCO images for backgrounds

download('http://images.cocodataset.org/zips/train2017.zip', 'train2017.zip')
download('http://images.cocodataset.org/zips/test2017.zip', 'test2017.zip')

In [ ]:
unzip('train2017.zip')
unzip('test2017.zip')

In [ ]:
os.rename('train2017', 'bg_train')
os.rename('test2017', 'bg_test')

In [ ]:
os.chdir('Data_Adobe')

In [ ]:
# composing foreground and background

!python compose.py --workers 8 --fg_path fg_train --mask_path mask_train --bg_path bg_train --out_path merged_train --out_csv Adobe_train_data.csv
!python compose.py --workers 8 --fg_path fg_test --mask_path mask_test --bg_path bg_test --out_path merged_test

In [ ]:
os.chdir('..')

### Supervised training

In [ ]:
# !python train_adobe.py -n Adobe_train -bs 4 -res 512
# !python train_adobe.py -n adobe_train -bs 1 -res 256
!python train_adobe.py -n adobe_train -bs 4 -res 128 -w 2 -epoch 10

### Data preparation for unsupervised training

In [ ]:
# splitting videos on frames
# get data here https://drive.google.com/drive/folders/1j3BMrRFhFpfzJAe6P2WDtfanoeSCLPiq?usp=sharing

!python frames_split.py -f Captured_Data/fixed-camera/train -e mp4

In [ ]:
# extracting masks

root = 'Captured_Data/fixed-camera/train'
for directory in (f'{root}/{path}' for path in os.listdir(root) if os.path.isdir(f'{root}/{path}')):
    !python test_segmentation_deeplab.py -i $directory

In [ ]:
# prepare real data

!python prepare_real.py

### Unsupervised training

In [ ]:
# !python train_real_fixed.py -n Real_fixed -bs 4 -res 512 -init_model models/adobe_train/net_epoch_60.pth
!python train_real_fixed.py -n real_fixed -bs 4 -res 128 -w 2 -epoch 10 -init_model models/adobe_train/net_epoch_10.pth